# Louvain Community Detection

The Louvain method of community detection is a greedy heirarical clsutering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score

*Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.*

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf
from scipy.io import mmread
from collections import OrderedDict

In [ ]:
# define a function to print the results
def print_parts(df, mod):
    
    #print the Modularity Score
    print('Modularity was {mod}')
    # See which nodes are in partition 0:
    part = []
    for i in range(len(df)):
        if (df['partition'][i] == 0):
            part.append(df['vertex'][i])
    print(part)

## Read data using cuDF

In [ ]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [ ]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()), )

In [ ]:
# create a Graph 
G1 = cugraph.Graph()
G1.add_edge_list(gdf["src"], gdf["dst"])

In [ ]:
# Call Louvain on the graph
df1, mod1 = cugraph.nvLouvain(G1) 

In [ ]:
print_parts(fd1, mod1)

## As a Matrix file using scipy

In [ ]:
# Read in the data file into scipy matrix format
mmFile='/datasets/networks/karate.mtx'
M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [ ]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)
G = cugraph.Graph()
G.add_adj_list(row_offsets, col_indices, values)

In [ ]:
# Call Louvain on the graph
df, mod = cugraph.nvLouvain(G) 

In [ ]:
# Check the modularity score
mod

In [ ]:
# See which nodes are in partition 0:
part = []
for i in range(len(df)):
    if (df['partition'][i] == 0):
        part.append(df['vertex'][i])
print(part)